In [1]:
import pandas as pd
import os
import re

def process_parquet(filename, working_dir, db_values=[15,10,5]):
    """
    Process a Parquet file, aggregate power by deadband (°F), and
    return a dict of Series like {'DB_15_120': <Series>, ...}.
    """
    file_path = os.path.join(working_dir, filename)
    df = pd.read_parquet(file_path)

    # Ensure datetime is rounded
    df['Time'] = df['Time'].dt.round('min')

    # Convert Celsius → Fahrenheit
    df['DB_F'] = df['Deadband_C'] * 9 / 5
    df = df.drop(columns=['Deadband_C'])


    results = {}

    # Extract the numeric part from "Shed120" for suffix
    match = re.search(r"ShedLong(\d+)", filename)
    suffix = match.group(1) if match else "122"

    for db in db_values:
        df_db = df[df['DB_F'] == db]
        if df_db.empty:
            # print(f"⚠️ No data found for {db}°F in {filename}")
            continue

        # print(df_db.columns)    

        power_sum = df_db.groupby('Time')['Water Heating Electric Power (kW)'].sum()
        upDB = df_db.groupby('Time')['Water Heating Deadband Upper Limit (C)'].mean()
        key = f"DB_{int(db)}_{suffix}"
        results[key] = power_sum
        # results[key] = {
        #     "power": power_sum,
        #     "upDB": upDB
        # }
    return results

print('Okay')

Okay


## import data

In [3]:
# -*- coding: utf-8 -*-
"""
3D Hot Water Outlet Temperature
Each control file is split by deadband along Y-axis
Median, 2.5th, and 97.5th percentile surfaces across houses
X-axis can be limited to a specific time window
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Patch



# ----------------- USER SETTINGS -----------------
WORKING_DIR = r"C:\Users\danap\OCHRE_Working"
CONTROL_FILES = [
    # "180110_1_3_ShedLong95_Control.parquet",
    # "180110_1_3_ShedLong100_Control.parquet",
    # "180110_1_3_ShedLong105_Control.parquet",
    # "180110_1_3_ShedLong110_Control.parquet",
    # "180110_1_3_ShedLong115_Control.parquet",
    # "180110_1_3_ShedLong120_Control.parquet",
    # "180110_1_3_ShedLong125_Control.parquet",
    "180113_1_3_EnergyShed_105_Control.parquet",
    "180113_1_3_EnergyShed_110_Control.parquet",
    "180113_1_3_EnergyShed_115_Control.parquet",
    "180113_1_3_EnergyShed_120_Control.parquet",
    "180113_1_3_EnergyShed_125_Control.parquet"
]
DEADBANDS = [15,10,5]   # deadbands to split along Y-axis
FILE_SPACING = 30          # spacing between files
DEADBAND_SPACING = 10      # spacing between deadbands within a file

TIME_COL = "Time"
TEMP_COL_C = "Hot Water Outlet Temperature (C)"
TEMP_COL_F = "Hot Water Outlet Temperature (F)"

# ----------------- X-AXIS WINDOW -----------------
x_min = 0    # start hour
x_max = 24   # end hour
t = 1

# ----------------- HELPER FUNCTION -----------------
def compute_percentiles_3d(filepath, deadband=None):
    df = pd.read_parquet(filepath)
    df[TIME_COL] = pd.to_datetime(df[TIME_COL])

    if deadband is not None and "Deadband" in df.columns:
        df = df[df["Deadband"] == deadband]

    df[TEMP_COL_F] = df[TEMP_COL_C] * 9/5 + 32

    df_ts = df.groupby(TIME_COL)[TEMP_COL_F].agg(
        P2_5=lambda x: np.percentile(x, 2.5),
        Median='median',
        P97_5=lambda x: np.percentile(x, 97.5)
    ).reset_index()
    return df_ts

# ----------------- GLOBAL TIME REFERENCE -----------------
all_start_times = []
for cf in CONTROL_FILES:
    df_tmp = pd.read_parquet(os.path.join(WORKING_DIR, cf))
    df_tmp[TIME_COL] = pd.to_datetime(df_tmp[TIME_COL])
    all_start_times.append(df_tmp[TIME_COL].min())
global_start_time = min(all_start_times)

# ----------------- LOAD ALL DATA -----------------
all_hours = None
Z_median = []
Z_low = []
Z_high = []
y_positions = []
y_labels = []

for file_idx, cf in enumerate(CONTROL_FILES):
    for db_idx, db in enumerate(DEADBANDS):
        df_ts = compute_percentiles_3d(os.path.join(WORKING_DIR, cf), deadband=db)
        if df_ts.empty:
            print(f"⚠️ No data for {cf}, deadband {db}. Skipping.")
            continue

        # Time relative to global start
        hours = (df_ts[TIME_COL] - global_start_time).dt.total_seconds()/3600
        if all_hours is None:
            all_hours = hours

        Z_median.append(df_ts['Median'].values)
        Z_low.append(df_ts['P2_5'].values)
        Z_high.append(df_ts['P97_5'].values)

        # Y-axis: combine file index + deadband spacing
        y_pos = file_idx*FILE_SPACING + db_idx*DEADBAND_SPACING
        y_positions.append(y_pos)

        # Correct Y-axis label without "Control"
        num = os.path.splitext(cf)[0].split('EnergyShed_')[-1].split('_Control')[0]
        y_labels.append(f"{num}_{db}")

# ----------------- CONVERT TO NUMPY ARRAYS -----------------
all_hours = np.array(all_hours)
Z_median = np.array(Z_median)
Z_low = np.array(Z_low)
Z_high = np.array(Z_high)
y_positions = np.array(y_positions)

# ----------------- FILTER X-AXIS WINDOW -----------------
mask = (all_hours >= x_min) & (all_hours <= x_max)
all_hours_filtered = all_hours[mask]
Z_median_filtered = Z_median[:, mask]
Z_low_filtered = Z_low[:, mask]
Z_high_filtered = Z_high[:, mask]

# ----------------- BUILD GRID -----------------
X = np.tile(all_hours_filtered, (len(Z_median_filtered), 1))
Y = np.tile(y_positions[:, None], (1, len(all_hours_filtered)))
Z_median = Z_median_filtered
Z_low = Z_low_filtered
Z_high = Z_high_filtered

print('Data Processed.')

Data Processed.


# plot data

In [134]:
%matplotlib qt

# ----------------- 3D SURFACE PLOT -----------------
fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111, projection='3d')

# Surface resolution
r = 9
c = 24 * 2

# ----------------- HIGHLIGHT SETUP -----------------
highlight_windows = [
    (6, 10),   # 6 AM → 10 AM
    (17, 20),  # 5 PM → 8 PM
]

def make_facecolors(base_color, shape, hilite_color,
                    intervals_per_hour, start_col):
    base_rgba      = np.array(plt.matplotlib.colors.to_rgba(base_color))
    highlight_rgba = np.array(plt.matplotlib.colors.to_rgba(hilite_color))

    nrows, ncols = shape
    fc = np.tile(base_rgba, (nrows, ncols, 1))

    for start_hr, end_hr in highlight_windows:
        start_idx = int(start_hr * intervals_per_hour) - start_col
        end_idx   = int(end_hr   * intervals_per_hour) - start_col

        start_idx = max(start_idx, 0)
        end_idx   = min(end_idx, ncols)

        if start_idx < end_idx:
            fc[:, start_idx:end_idx, :] = highlight_rgba

    return fc

# Base + highlight colors
low_base, low_hi = 'plum', 'orchid'
med_base, med_hi = 'lightgreen', 'xkcd:leaf green'
hi_base,  hi_hi  = 'xkcd:goldenrod', 'darkorange'

# ----------------- SURFACE PLOTS -----------------

# --- define intervals_per_hour & start_col ---
intervals_per_hour = X.shape[1] // 24  # e.g., 24 columns = 1 per hour
start_col = 0

# --- plot median surface ---
ax.plot_surface(
    X, Y, Z_median,
    facecolors=make_facecolors(
        med_base, X.shape, med_hi,
        intervals_per_hour, start_col
    ),
    edgecolor='black',
    linewidth=1,
    shade=False,
    rcount=r,
    ccount=c
)

ax.plot_surface(
    X, Y, Z_median,
    facecolors=make_facecolors(
        med_base, X.shape, med_hi,
        intervals_per_hour, start_col
    ),
    edgecolor='black',
    linewidth=1,
    shade=False,
    rcount=r,
    ccount=c
)

ax.plot_surface(
    X, Y, Z_low,
    facecolors=make_facecolors(
        low_base, X.shape, low_hi,
        intervals_per_hour, start_col
    ),
    edgecolor='black',
    linewidth=1,
    shade=False,
    rcount=r,
    ccount=c
)

ax.plot_surface(
    X, Y, Z_high,
    facecolors=make_facecolors(
        hi_base, X.shape, hi_hi,
        intervals_per_hour, start_col
    ),
    edgecolor='black',
    linewidth=1,
    shade=False,
    rcount=r,
    ccount=c
)

# ----------------- AXES FORMATTING -----------------
ax.set_xlabel("Time [H]", fontsize=14, labelpad=10)
ax.set_ylabel("Tshed_TDB [°F]", fontsize=14, labelpad=20)
ax.set_zlabel("Outlet Temp [°F]", fontsize=14, labelpad=10)
ax.view_init(elev=25, azim=-50)

B = 12
ax.set_xticks(np.arange(x_min, x_max + 1, t+1))
ax.set_xticklabels(
    [f"{int(h):02d}" for h in np.arange(x_min, x_max + 1, t+1)],
    fontsize=B
)

ax.set_ylim(y_positions.min(), y_positions.max())
ax.tick_params(axis='y', pad=-5)
ax.set_yticks(y_positions)
ax.set_yticklabels(y_labels, rotation=-45, ha='left', va='top', fontsize=10)

ax.yaxis._axinfo['grid'].update(
    color='black', linestyle=':', linewidth=0.5
)

ax.set_zlim(90, 140)
ax.tick_params(axis='z', labelsize=B)

zticklabels = ax.get_zticklabels()
if zticklabels:
    zticklabels[0].set_visible(False)

# ----------------- LEGEND -----------------
legend_elements = [
    Patch(facecolor=hi_base,  edgecolor='black', label='97.5%'),
    Patch(facecolor=med_base, edgecolor='black', label='Median'),
    Patch(facecolor=low_base, edgecolor='black', label='2.5%')
]

ax.legend(
    handles=legend_elements,
    bbox_to_anchor=(0.84, 0.02),
    ncol=3,
    fontsize=14,
    frameon=False
)

# ----------------- SAVE / SHOW -----------------
RESULTS_DIR = r"C:\Users\danap\OCHRE_Working\Figs\180110_1_3_ShedLong_3DTout_95CI.pdf"
# plt.savefig(RESULTS_DIR, format='pdf', bbox_inches='tight', pad_inches=1)
plt.show()


In [83]:
import os
import re
import pandas as pd

def aggregate_across_deadbands(work_dir, prefix):
    """
    Aggregate files like:
      <prefix>_Control_DB5.parquet
      <prefix>_Control_DB10.parquet
      <prefix>_Control_DB15.parquet

    Into:
      <prefix>_Control.parquet
    """

    pattern = re.compile(
        rf"^{re.escape(prefix)}_Control_DB(\d+)\.parquet$"
    )

    matches = []
    for fname in os.listdir(work_dir):
        m = pattern.match(fname)
        if m:
            matches.append((fname, int(m.group(1))))

    if not matches:
        print(f"⚠️ No deadband files found for prefix {prefix}")
        return

    dfs = []
    for fname, dbF in sorted(matches, key=lambda x: x[1]):
        path = os.path.join(work_dir, fname)
        df = pd.read_parquet(path)

        # Ensure deadband info is present
        df["Shed Deadband (F)"] = dbF
        df["SourceFile"] = fname

        dfs.append(df)

    df_all = pd.concat(dfs, ignore_index=True)

    out_path = os.path.join(work_dir, f"{prefix}_Control.parquet")
    df_all.to_parquet(out_path, index=False)

    print(
        f"✅ Aggregated {len(matches)} deadbands "
        f"({', '.join(str(db) for _, db in matches)})\n"
        f"   → {out_path}\n"
        f"   Rows: {len(df_all):,}"
    )


aggregate_across_deadbands(
    work_dir=WORKING_DIR,
    prefix="180113_1_3_EnergyShed_120"
)


✅ Aggregated 3 deadbands (10, 15, 5)
   → C:\Users\danap\OCHRE_Working\180113_1_3_EnergyShed_120_Control.parquet
   Rows: 587,520
